# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.29/10670/1 Dashboard: http://10.20.0.29:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1008,Patricia,0.15864472106038807,0.9812639644999739
2000-01-01 00:00:01,1008,Quinn,-0.4126941258179131,0.4203676743717186
2000-01-01 00:00:02,1067,Laura,0.4535217643829417,-0.5306091691946111
2000-01-01 00:00:03,996,Sarah,0.3730665706802092,-0.2805804812925936
2000-01-01 00:00:04,1006,Victor,-0.034958764953235466,0.4285228268018819
2000-01-01 00:00:05,1016,Michael,-0.7525136278832607,0.8630818092750212
2000-01-01 00:00:06,1035,Victor,-0.8839599244134992,-0.320766615490806
2000-01-01 00:00:07,1010,Hannah,0.8943454583497881,-0.18104494559906015
2000-01-01 00:00:08,967,Ingrid,0.4897953274656761,0.9719138041728912


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,976,Kevin,0.09357418699164488,-0.9053659669393135
2000-01-30 00:00:01,1035,Hannah,0.17306525311006804,0.7134161812081383
2000-01-30 00:00:02,1008,Xavier,0.9075295114097659,0.8372240248569134
2000-01-30 00:00:03,997,Kevin,0.9215202812973804,-0.3760860377300812
2000-01-30 00:00:04,1041,Tim,0.23835615535432653,-0.5653024786717102
2000-01-30 00:00:05,1037,Laura,0.9624217527679262,0.2736877802612838
2000-01-30 00:00:06,959,Laura,0.6161362614673098,0.20051029156609568
2000-01-30 00:00:07,1022,Tim,0.4180747755458363,0.8072654001168373
2000-01-30 00:00:08,973,Zelda,0.6730613511586587,0.48907780151975455


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1008,Patricia,0.158645,0.981264
1,2000-01-01 00:00:01,1008,Quinn,-0.412694,0.420368
2,2000-01-01 00:00:02,1067,Laura,0.453522,-0.530609
3,2000-01-01 00:00:03,996,Sarah,0.373067,-0.280580
4,2000-01-01 00:00:04,1006,Victor,-0.034959,0.428523


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1008,Patricia,0.158645,0.981264
1,2000-01-01 00:00:01,1008,Quinn,-0.412694,0.420368
2,2000-01-01 00:00:02,1067,Laura,0.453522,-0.530609
3,2000-01-01 00:00:03,996,Sarah,0.373067,-0.280580
4,2000-01-01 00:00:04,1006,Victor,-0.034959,0.428523


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.46 s, sys: 576 ms, total: 5.04 s
Wall time: 3.26 s


name
Alice      -0.000248
Bob         0.001779
Charlie     0.001204
Dan        -0.001487
Edith      -0.000077
Frank       0.000306
George      0.003272
Hannah      0.000319
Ingrid      0.000977
Jerry       0.001534
Kevin       0.000038
Laura       0.003747
Michael    -0.001695
Norbert     0.000770
Oliver     -0.000202
Patricia    0.003887
Quinn      -0.003282
Ray         0.002201
Sarah      -0.001342
Tim        -0.000890
Ursula     -0.000344
Victor     -0.000660
Wendy      -0.000872
Xavier      0.000632
Yvonne     -0.000476
Zelda      -0.000115
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.36 s, sys: 149 ms, total: 1.51 s
Wall time: 1.03 s


name
Alice      -0.000248
Bob         0.001779
Charlie     0.001204
Dan        -0.001487
Edith      -0.000077
Frank       0.000306
George      0.003272
Hannah      0.000319
Ingrid      0.000977
Jerry       0.001534
Kevin       0.000038
Laura       0.003747
Michael    -0.001695
Norbert     0.000770
Oliver     -0.000202
Patricia    0.003887
Quinn      -0.003282
Ray         0.002201
Sarah      -0.001342
Tim        -0.000890
Ursula     -0.000344
Victor     -0.000660
Wendy      -0.000872
Xavier      0.000632
Yvonne     -0.000476
Zelda      -0.000115
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.29 s, sys: 115 ms, total: 1.41 s
Wall time: 1.03 s


name
Alice      -0.000248
Bob         0.001779
Charlie     0.001204
Dan        -0.001487
Edith      -0.000077
Frank       0.000306
George      0.003272
Hannah      0.000319
Ingrid      0.000977
Jerry       0.001534
Kevin       0.000038
Laura       0.003747
Michael    -0.001695
Norbert     0.000770
Oliver     -0.000202
Patricia    0.003887
Quinn      -0.003282
Ray         0.002201
Sarah      -0.001342
Tim        -0.000890
Ursula     -0.000344
Victor     -0.000660
Wendy      -0.000872
Xavier      0.000632
Yvonne     -0.000476
Zelda      -0.000115
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html